In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# on kaggle
import numpy as np #
import pandas as pd #

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filepath = (os.path.join(dirname, filename))

In [ ]:
# #on local 
# filepath = '../drivedata/coursea_data.csv'

In [ ]:
# # on Google drive
# filepath = "/content/drive/My Drive/Colab Notebooks/coursera/data/coursea_data.csv"

In [ ]:
df = pd.read_csv(filepath, index_col=0)

## Items
- course_title : Contains the course title.
- course_organization : It tells which organization is conducting the courses.
- courseCertificatetype : It has details about what are the different certifications available in courses.
- course_rating : It has the ratings associated with each course.
- course_difficulty : It tells about how difficult or what is the level of the course.
- coursestudentsenrolled : It has the number of students that are enrolled in the course.

## Check roughly

In [ ]:
from collections import defaultdict
dct_convert_unit = defaultdict(lambda:0)
dct_convert_unit['k']  = 1000
dct_convert_unit['m']  = 1000*1000

df['course_students_enrolled_unit'] = df['course_students_enrolled'].str[-1].apply(lambda x : dct_convert_unit[x])
df['course_students_enrolled'] = df['course_students_enrolled'].str[:-1].astype(float)
df['course_students_enrolled'] = df['course_students_enrolled'] * df['course_students_enrolled_unit'] 
del df['course_students_enrolled_unit'] 

In [ ]:
ls_course_Certificate_type = ['COURSE','SPECIALIZATION' ,'PROFESSIONAL CERTIFICATE']
ls_course_difficulty = [ 'Beginner', 'Intermediate', 'Advanced','Mixed']

In [ ]:
df.shape[0]

In [ ]:
for _col  in df.columns:
    if df[_col].dtype == 'object':  
        display(pd.DataFrame(df[_col].value_counts()))

In [ ]:
for _col  in df.columns:
    if df[_col].dtype == 'object':  
        display(pd.DataFrame(df[_col].value_counts(normalize=True)*100).cumsum())

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.sort_values(by="course_students_enrolled",ascending=True)

## Check categorical features


### certification type and difficulty
- Course > Sepcial >> Pro
  - Ex. 
      - Course : *Marchine Learning*
      - SP : *Deep Learning*
      - Pro : *IBM Data Science* 
  - <u>Maybe related to difficulty?</u>
- Certification type and difficulty are linked?
  - Heat map:
    - Heat map prove below.

In [ ]:
for _col  in ['course_Certificate_type','course_difficulty']:   
    fig, ax = plt.subplots(figsize=(8,6), facecolor='w')
    sns.countplot(x=_col, data=df, ax= ax, order = df[_col].value_counts().index, alpha=0.7)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6), facecolor='w')
# sns.countplot(x='course_Certificate_typea', hue ='course_difficulty', order = ls_course_Certificate_type,hue_order=ls_course_difficulty,  data=df, ax= ax, alpha=0.7)
# ax.legend(loc='upper right')
df_pivot_cert_difficult = df[['course_Certificate_type', 'course_difficulty']].pivot_table(
        index='course_difficulty', columns='course_Certificate_type',
        aggfunc=len, fill_value=0)
df_pivot_cert_difficult = df_pivot_cert_difficult.loc[ls_course_difficulty,  ls_course_Certificate_type]
sns.heatmap(df_pivot_cert_difficult, annot=True, fmt="1.1f",cmap="Reds")


### Rating distribute

- course_difficulty
  - Just by looking, difficulty related to rating. 
    - Ex. this course is soooo difficult! I hate this course... orz 
  - But mean and std in each difficulty are almost same.
    - it mean organizer set up course content cocommensurate with difficulty in many course
    - 0.1 of distance is large different???
- course_Certficate
  - Same as below
    - But a #Pro is very few. I cannot affirm this analysis


In [ ]:
fig, ax = plt.subplots(figsize=(8,6), facecolor='w')
sns.countplot(x='course_rating', data=df, ax= ax,hue = 'course_difficulty', order = sorted(list(set(df.course_rating))), hue_order=ls_course_difficulty, alpha=0.7)
ax.legend()
df_rating_dist =  df[['course_rating', 'course_difficulty']] \
    .pivot_table(
        columns='course_difficulty', index='course_rating',
        aggfunc=len, fill_value=0)

display(df_rating_dist.T)


df_rating_dist = 100*df_rating_dist/df_rating_dist.sum()
fig, ax = plt.subplots(figsize=(8,6), facecolor='w')
ls_xticklabel = list(df_rating_dist.index)
for i , _col in enumerate(ls_course_difficulty):
    ax.bar(
        list(map(lambda x: (0.2)*i+x, range(len((ls_xticklabel))))), 
        height=df_rating_dist[_col].values,
        label=_col,
        alpha=0.6, width=0.2)
ax.set_xticks(range(len((ls_xticklabel))))
ax.set_xticklabels(ls_xticklabel)
plt.xlabel('coruse_rating')
plt.ylabel('coruse_rating')
ax.legend()

df_rating_dist =  df[['course_rating', 'course_difficulty']] \
  .groupby('course_difficulty')['course_rating'].agg(["mean","std"])
display(df_rating_dist)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6), facecolor='w')
sns.countplot(x='course_rating', data=df, ax= ax,hue = 'course_Certificate_type', order = sorted(list(set(df.course_rating))), hue_order=ls_course_Certificate_type, alpha=0.7)
ax.legend()
df_rating_dist =  df[['course_rating', 'course_Certificate_type']] \
    .pivot_table(
        columns='course_Certificate_type', index='course_rating',
        aggfunc=len, fill_value=0)

display(df_rating_dist.T)


df_rating_dist = 100*df_rating_dist/df_rating_dist.sum()
fig, ax = plt.subplots(figsize=(8,6), facecolor='w')
ls_xticklabel = list(df_rating_dist.index)
for i , _col in enumerate(ls_course_Certificate_type):
    ax.bar(
        list(map(lambda x: (0.2)*i+x, range(len((ls_xticklabel))))), 
        height=df_rating_dist[_col].values,
        label=_col,
        alpha=0.6, width=0.2)
ax.set_xticks(range(len((ls_xticklabel))))
ax.set_xticklabels(ls_xticklabel)
plt.xlabel('coruse_rating')
plt.ylabel('count[%]')
ax.legend()

df_rating_dist =  df[['course_rating', 'course_Certificate_type']] \
  .groupby('course_Certificate_type')['course_rating'].agg(["mean","std"])
display(df_rating_dist)


### course_students_enrolled
- course_students_enrolled distribution is long tail
  - convet log scale
- Break down analysis same as below
  - couse = SP < Pro
    - #Pro is few.　But Many pro course have many students when looking each data      
  - #student is not different between type of difficulty
  - But  I want to know "MIX", what is meaning of MIX?

In [ ]:
fig, ax = plt.subplots(figsize=(8,6), facecolor='w')
_cert_type = 'PROFESSIONAL CERTIFICATE'
sns.distplot(df.loc[:, "course_students_enrolled"],ax=ax, label = _cert_type)

In [ ]:
_cert_type

In [ ]:
df_large = df[df.course_students_enrolled > 500000] \
  .sort_values(by="course_students_enrolled", ascending =False)
print(df_large.shape[0])
display(df_large)

In [ ]:
df_small = df[df.course_students_enrolled < 10000]
print(df_small.shape[0])
display(df_small)

In [ ]:
import copy

df['log_course_students_enrolled'] = np.log10(df.course_students_enrolled)

In [ ]:
display(pd.DataFrame(df.groupby('course_Certificate_type')['log_course_students_enrolled'].agg(['mean','std','count','min','max'])))
display(df.loc[df.course_Certificate_type == "PROFESSIONAL CERTIFICATE", ["course_title","course_organization","course_students_enrolled"]])
fig, ax = plt.subplots(figsize=(8,6), facecolor='w')
for _cert_type in ls_course_Certificate_type:
  sns.kdeplot(df.loc[df.course_Certificate_type == _cert_type, "log_course_students_enrolled"],ax=ax, label = _cert_type)
# plt.xlim(-10000,500000)
plt.xlabel('log course_students_enrolled')
plt.ylabel('p')



In [ ]:
display(pd.DataFrame(df.groupby('course_difficulty')['course_students_enrolled'].agg(['mean','std','count','min','max'])))
fig, ax = plt.subplots(figsize=(8,6), facecolor='w')
for _diff in ['Beginner', 'Intermediate', 'Mixed', 'Advanced']:
  sns.kdeplot(df.loc[df.course_difficulty == _diff, "course_students_enrolled"],ax=ax, label = _diff)
# plt.xlim(-10000,500000)
plt.legend()
plt.xlabel('course_students_enrolled')
plt.ylabel('p')

### course_organization
- course_organization distribution  has long tail
- Focus in # of held 
  - Classify 2 class ... high vs low
    - high held orgnizer got more users than lower class

In [ ]:
_col ="course_organization"
fig, ax = plt.subplots(figsize=(8,6), facecolor='w')
sns.countplot(x=_col, data=df, order=df[_col].value_counts().index, ax= ax, alpha=0.7)
df_org = pd.DataFrame(df[_col].value_counts())
mean_part = df_org.mean().values[0]
print("mean: {0}".format(mean_part))
ax.hlines(y=mean_part, xmin=0, xmax=ax.get_xlim()[1], color='r')
ax.set(xlim=(0,70))
plt.xticks(rotation= 90)

In [ ]:
df_org = df_org.reset_index()
df_org.columns = ['course_organization', 'n_held']
df_org['course_organization_type'] = 'low_held'
df_org.loc[df_org['n_held']>mean_part,'course_organization_type']  = 'high_held'

In [ ]:
df_with_held = pd.merge(df  , df_org, on='course_organization', how='left')
sns.pairplot(df_with_held,hue='course_organization_type',palette="husl")

In [ ]:
print("course_students_enrolled")
df_with_held.groupby(['course_organization_type'])['course_students_enrolled'].agg(['mean','count','min','max'])

### course_title
- titles in these course has many languages.
  - EN, RU, and so on
  - => we detect langeage of title and normarize it
  - Language detection using "cld2" is not correct completely. (Off course, theis package is amazing and very usefull!)
    - -> I pick up ONLY english and analyze. 
- As following graph, these words are most frequent words in title,
  - I deem that these words has some class, for instance programming, biz and so on ..., if you have time to analyze more, could you try LDA?
  - Finally, I pick up words which appear more than 20title, I define these words as "top words",  then how many top words does title include? and how does #top words effect #user?
    - Uhh... top words have power to attract user. 

In [ ]:
!pip install gensim
!pip install -q wordcloud
!pip install pycld2
import wordcloud
import nltk
import gensim
from gensim import corpora
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
import unicodedata
import re
import string
import pycld2 as cld2

In [ ]:
"""
Set up NLP
"""

# Constants
# POS (Parts Of Speech) for: nouns, adjectives, verbs and adverbs
DI_POS_TYPES = {'NN':'n', 'JJ':'a', 'VB':'v', 'RB':'r'} 
POS_TYPES = list(DI_POS_TYPES.keys())

# Constraints on tokens
MIN_STR_LEN = 3
RE_VALID = '[a-zA-Z]'

# Get stopwords, stemmer and lemmatizer
stopwords = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

# Remove accents function
# (Normalization Form Compatibility Decomposition)
def remove_accents(data):
    return ''.join(x for x in unicodedata.normalize('NFKD', data) if x in string.ascii_letters or x == " ")

def tokenize(sentence):
    # Tokenize by sentense:
    tokens = [word.lower() for sent in nltk.sent_tokenize(sentence) for word in nltk.word_tokenize(sent)]
    ls_tokenized = []
    for token in tokens:

        # Remove accents
        t = remove_accents(token)

        # Remove punctuation
        t = str(t).translate(string.punctuation)

        #remove stop word
        if t in stopwords:
            continue

        # remove signal etc...
        if not re.search(RE_VALID, t):
            continue

        # #remove too short
        # if len(t) < MIN_STR_LEN:
        #   continue

        #stem and lemma
        # t = stemmer.stem(t)
        # t = lemmatizer.lemmatize(t)#, pos=DI_POS_TYPES[])

        ls_tokenized.append(t)
    return ls_tokenized

In [ ]:
# Detect monther language of title
df['course_title_lang'] = [cld2.detect(_title)[2][0][1]  for _title in df.course_title.values]

In [ ]:
print("course_students_enrolled")
df.groupby(['course_title_lang'])['course_students_enrolled'].agg(['mean','count','min','max']).sort_values(by='count')

In [ ]:
df_en = df[df.course_title_lang=='en']
dct_title = {i :tokenize(_token) for i, _token in enumerate(df_en.course_title.values)}
df_words = pd.DataFrame(nltk.FreqDist([flatten for inner in dct_title.values() for flatten in inner]),index=['count']).T.sort_values(by='count',ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(20,6), facecolor='w')
df_words[df_words['count']>20].plot.bar(ax=ax)
plt.xticks(rotation =75,fontsize=16)
plt.xlabel('top appear words')
plt.ylabel('count')
print(df_words.shape[0])

In [ ]:
ls_top_words = list(df_words[df_words['count']>20].index)

In [ ]:
df_top_words=pd.DataFrame(
    {i :len(list(set(ls_top_words) & set(ls_words) )) for i , ls_words in enumerate(dct_title.values())},
    index=['top_w_count']).T

In [ ]:
df_with_top_words = pd.concat([df_en.reset_index(drop=True),df_top_words],axis=1)

In [ ]:
_col = 'top_w_count'
fig, ax = plt.subplots(figsize=(20,6), facecolor='w')
df_plot = df_with_top_words.groupby([_col])['course_students_enrolled'].agg(['mean']).reset_index()
sns.regplot(data=df_plot, x='top_w_count', y='mean',ax=ax,scatter_kws={"s": 80, "color":"r"}, robust=True, ci=None)
plt.xticks(rotation =0,fontsize=16)
plt.xlim(-0.5,6.5)
plt.xlabel('#top_word in title')
plt.ylabel('mean #course_students_enrolled')

# My 6hour(3day*2h) challenge : TIme out  